In [11]:
import pandas as pd
names = []
with open('columns.txt', 'r') as f:
    for line in f:
        names.append(line[:-1])

In [12]:
names

['Scrap_date',
 'Scrap_time',
 'Country_from',
 'Country_to',
 'Flight_id',
 'Flight_date',
 'Airline',
 'Change',
 'Price',
 'There_back',
 'Depart_hour',
 'Journey_from',
 'Depart_from',
 'Arrival_hour',
 'Journey_to',
 'Arrive_to']

In [13]:
import pandas as pd
dat = pd.DataFrame(columns = names)

In [14]:
dat

,Scrap_date,Scrap_time,Country_from,Country_to,Flight_id,Flight_date,Airline,Change,Price,There_back,Depart_hour,Journey_from,Depart_from,Arrival_hour,Journey_to,Arrive_to


Dowolny filtr na podstawie pliku json. Tutaj trzeba dodać możliwość typu porównania dla wartości ilościowych, typu "bigger", "equal", "less".

In [ ]:
from pyspark import SparkContext, SparkConf
import json

filepath = ''

def custom_filter_generator(conf_path):
    with open(conf_path) as json_file:  
        configuration = json.load(json_file)
    config = list(configuration.values())
    indices = [i for i in range(len(config)) if config[i] != 'None']
    return lambda x :  sum([x.split(';')[i] == config[i] for i in indices]) 
        
    
conf = SparkConf().setAppName("flights").setMaster("local[*]")
sc = SparkContext(conf = conf)
flightsRDD = sc.textFile(filepath)
header = flightsRDD.first()
flightsRDD = flightsRDD.filter(lambda line: line != header)
filer_rule = custom_filter_generator('conf.txt')
routeRDD = flightsRDD.filter(filter_rule)

Wyszukiwanie najtańszych połączeń dla zadanego filtra. Znajduje połączenie o najniższej cenie dla podanych parametrów filtrowania.

In [ ]:
prices = peopleRDD.flatMap(lambda line: float(line.split(';')[8]))
cheapest = prices.reduce(lambda x, y: min(x,y))
cheapest_route = fligthsRDD.filter(lambda line : float(line.split(';')[8]) == cheapest)
cheapest_route.saveAsTextFile('\out\filtered.text')

W jakie dni i w jakich porach są najtańsze bilety dla danego połączenia (kraj - kraj). Analogicznie można robić z departure_from oraz departure_to. To samo przetwarzanie można zastosować przy sprawdzaniu jak zmieniały się ceny przewoźnika na danej linii. Tutaj zczytujemy tylko najniższą cenę podróży ( w obie strony), nie patrzymy jak daleko jest lot od daty dzisiejszej.

In [ ]:
from statistics import mean
from matplotlib import pyplot as plt

conf = SparkConf().setAppName("flights").setMaster("local[*]")
sc = SparkContext(conf = conf)

country_from = 'France'
country_to = 'Denmark'

filepaths =  #lista list sciezek do plikow / nazw w kolejnosci : pon_0, pon_1, pon_2, pon_3, wt_0, wt_1, itd.
mean_prices = []
for filepath in filepaths:
    flightsRDD = sc.textFile(filepath)
    header = flightsRDD.first()
    filtered = flightsRDD.filter(lambda line: line != header)
    filtered = filtered.filter(lambda line: line.split(';')[2] == country_from and line.split(';')[3] == country_to)
    pairRDD = filtered.map(lambda line: (float(line.split(';')[4]), float(line.split(';')[8])))
    groupedByJourneyID = pairRDD.groupByKey()
    pricesByJourney = groupedByJourneyID.reduceByKey(lambda x,y : x + y)
    mean_prices.append(pricesByJourney.mapValues(lambda price: price).reduce(lambda x,y : mean(x,y)))

# Tutaj jest przetwarzanie dla jednego tygodnia, oczywiscie zrobi sie dla kazdego, natomiast to zalezy jak te pliki beda
# wygladaly


Można analizować ceny biletów np. na danej linii, albo nawet na wszystkich liniach, ale moim zdaniem jak robimy analizę to jednak jesteśmy zazwyczaj zainteresowani jakimiś konkretnymi połączeniami. Jako klienci linii lotniczych, nie interesuje nas trend kiedy ceny w ogólności są najtańsze, ale kiedy są najtańsze na danej linii. Albo szukamy okazji i kupujemy jak coś się pojawi ad hoc.

In [ ]:
# Liczy srednia cene podrozy (w dwie strony) pomiedzy dwoma krajami dla zadanego okresu wyprzedzenia kupna (okreslonego,
# jako liczba dni pomiedzy sprawdzeniem ceny, a data odlotu). Niezaleznie od dnia i pory kupna biletu.

from statistics import mean
from matplotlib import pyplot as plt
import datetime
import numpy as np

conf = SparkConf().setAppName("flights").setMaster("local[*]")
sc = SparkContext(conf = conf)

country_from = 'France'
country_to = 'Denmark'
time_from_check_to_flight = 8

def daytime_filter_generator(days):
    def daytime_filter(line: str):
        time_from_check_to_flight = days
        line = line.split(';')
        scrap_day = datetime.datetime.strptime(line[0], '%Y-%m-%d')
        flight_day = datetime.datetime.strptime(line[5], '%d/%m/%y')
        return abs(flight_day - scrap_day) == datetime.timedelta(time_from_check_to_flight) and line[9] == "There"
    return daytime_filter

filepaths =  #lista list sciezek do plikow / nazw w kolejnosci : pon_0, pon_1, pon_2, pon_3, wt_0, wt_1, itd.
p = 0
n = 0
for filepath in filepaths:
    flightsRDD = sc.textFile(filepath)
    header = flightsRDD.first()
    flightsRDD = flightsRDD.filter(lambda line: line != header)
    filtered = flightsRDD.filter(lambda line: line.split(';')[2] == country_from and line.split(';')[3] == country_to)
    pairRDD = filtered.map(lambda line: (float(line.split(';')[4]), float(line.split(';')[8])))
    groupedByJourneyID = pairRDD.groupByKey()
    pricesByJourney = groupedByJourneyID.reduceByKey(lambda x,y : x + y)
    daytime_filter = daytime_filter_generator(time_from_check_to_flight)
    day_filtered = flightsRDD.filter(daytime_filter).map(lambda line: line.split(';')[4]).collect()
    day_filtered = list(unique(np.array(day_filtered)))
    for id_j , price in pricesByJourney.collect():
        if id_j in day_filtered:
            p += price
    n += len(day_filtered)
print("Mean price for {}-{} journey with {} days booking before flight: {}" \
      .format(country_from, country_to, time_from_check_to_flight, p / n))